In [ ]:
from pydantic import BaseModel, EmailStr
from typing import List, Dict, Optional

You will have to write lot's of check statements in the code for the variables to store data correctly and how you want. Pydantic solves that problem.

In [ ]:
def insert_patient_details(name: str, age: int):

  if type(name) == str and type(age) == int:
    if age <0 :
      raise ValueError("Age can't be negative")
    else:
      print(name)
      print(age)
      print('inserted')
  else:
    raise TypeError("Incorrect data type")

def update_patient_details(name: str, age: int):

  if type(name) == str and type(age) == int:
    if age <0 :
      raise ValueError("Age can't be negative")
    else:
      print(name)
      print(age)
      print("updated")
  else:
    raise TypeError("Incorrect data type")

In [ ]:
patient_info = {'name': "Arjun", "age": 28}

In [ ]:
Patient_1_insert = insert_patient_details(**patient_info)
Patient_1_update = update_patient_details(**patient_info)

Arjun
28
inserted
Arjun
28
updated


Using Pydantic.

1. Define a Pydantic model (class) that represents the ideal schema of the data.
* This includes the expected fields, their types, and any validation constraints (e.g., gt =0 for positive numbers)

* These all are fields (name, age, weight, ......).

In [ ]:
class Patient(BaseModel):

  name: str
  age: int
  weight: float
  married: bool
  allergies: List[str]
  contact_details: Dict[str, str]

2. Instantiate the model with raw input data (usually a dictionary or JSON-like structure).
* Pydantic will automatically validate the data and coerce it into the correct Python types (if possible).
* If the data doesn't meet the model's requirements, Pydantic raises a Validation Error.

In [ ]:
patient_info = {"name": "Arjun", "age": 30, 'weight': 72.0, "married": False, "allergies": ['nuts', 'milk'], "contact_details": {'email': 'ar@gmail.com', 'phone': '7893647'}}
Patient_1 = Patient(**patient_info)

3. Pass the validated model object to functions or use it throughout your codebase.
* This ensures that every part of your program works with clean, type-safe, and logically valid data.

In [ ]:
def insert_patient_details(patient: Patient):
  print(patient.name)
  print(patient.age)
  print("Inserted")

def update_patient_details(patient: Patient):
  print(patient.name)
  print(patient.age)
  print("Updated")

In [ ]:
insert_patient_details(Patient_1)

Arjun
30
Inserted


Solving data validation problem.
* Restructing your data type.

1. Adding default values to the variables.
* Use Optional from typing module and also giving it default value of None.

In [ ]:
class Patient(BaseModel):

  name: str
  age: int
  weight: float
  married: Optional[bool] = False
  allergies: Optional[List[str]]= None
  contact_details: Dict[str, str]

In [ ]:
!pip install email_validator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 11.6 MB/s eta 0:00:00


Data validation.
1. Custom data type from Pydantic: Email validation using EmailStr. AnyUrl to insert the correct Urls.

In [ ]:
from pydantic import BaseModel, EmailStr, AnyUrl
from typing import List, Dict, Optional

class Patient(BaseModel):

  name: str
  email: Optional[EmailStr] = None
  website: Optional[AnyUrl] = None
  age: int
  weight: float
  married: Optional[bool] = False
  allergies: Optional[List[str]]= None
  contact_details: Dict[str, str]

def insert_patient_details(patient: Patient):
  print(patient.name)
  print(patient.age)
  print("Inserted")

def update_patient_details(patient: Patient):
  print(patient.name)
  print(patient.age)
  print("Updated")

patient_info = {"name": "Arjun", 'email': 'ar@gmail.com', "age": 30,
                'weight': 72.0, "married": False, "allergies": ['nuts', 'milk'],
                "contact_details": {'zip': '11800', 'phone': '7893647'}}

Patient_1 = Patient(**patient_info)

insert_patient_details(Patient_1)

Arjun
30
Inserted


2. Custom data validation using "Field" function. So no one can insert age as negative.
* You can also attach meta data. So others can understand it. Using Annotated from typing module.
* Setting default values.
* Using strict parameter which stops type conversions. For eg: If you set the value of weigth as str "72". The Pydantic module will still work. It will not show any errors evenif you tell it that it should be float. But some time it is not good. So, to stop that you can use the "strict" parameter in the Field.

In [ ]:
from pydantic import BaseModel, EmailStr, AnyUrl, Field
from typing import List, Dict, Optional, Annotated

class Patient(BaseModel):

  #name: str = Field(max_length=50)  # max name length 50
  name: Annotated[str, Field(max_length = 50, title = "Name of the patient", description="Give the name of the patient in less than 50 chars",
                             examples = ["Arjun", "Dignon"])]
  email: Optional[EmailStr] = None
  website: Optional[AnyUrl] = None
  age: int = Field(gt=0, lt=120)    # Greater than zero and less then 120.
  weight: Annotated[float, Field(ge=1, strict=True)]       # Greater than and equal to 1. And also it should be float.
  married: Optional[bool] = False
  allergies: Optional[List[str]] = Field(default = None, max_length=8)  # No one can add more than 8 allergies and default is None.
  contact_details: Dict[str, str]

def insert_patient_details(patient: Patient):
  print(patient.name)
  print(patient.age)
  print("Inserted")

def update_patient_details(patient: Patient):
  print(patient.name)
  print(patient.age)
  print("Updated")

patient_info = {"name": "Arjun", 'email': 'ar@gmail.com', "age": 30,
                'weight': 72.0, "married": False, "allergies": ['nuts', 'milk'],
                "contact_details": {'zip': '11800', 'phone': '7893647'}}

Patient_1 = Patient(**patient_info)

insert_patient_details(Patient_1)

Arjun
30
Inserted


Field validator.
1. To check if the email is from one company or some other company. Like if you give some discounts to some company employess and to check if that employee works at that company or not. Like the employee who are working at PwC should have @pwc in their emails.
2. Transformation. Convert the name in upper case.
3. Field validator can be operate in two modes: before and after. Here by default it is after.

In [ ]:
from pydantic import BaseModel, EmailStr, AnyUrl, field_validator
from typing import List, Dict, Optional

class Patient(BaseModel):

  name: str
  email: Optional[EmailStr] = None
  website: Optional[AnyUrl] = None
  age: int
  weight: float
  married: Optional[bool] = False
  allergies: Optional[List[str]]= None
  contact_details: Dict[str, str]

  @field_validator('email')
  @classmethod
  def email_validator(cls, value):

    valid_domais = ['pwc.com', 'kymeratx.com']
    # extracting domain name from the value.
    domain_name = value.split('@')[1]    # ['arjun','pwc.com']
    if domain_name not in valid_domais:
      raise ValueError('Not a valid domain')

  @field_validator('name')
  @classmethod
  def transform_name(cls, value):
    return value.upper()

  # You can also do that using Field function. Here we want to set the limit of age usng Field validator.
  @field_validator('age', mode = 'after')        # Here by setting after means that the age will first pass through class and it will convert from str to int. But if you use "before" it will use the age actual value which is in str. Which will give error.
  @classmethod
  def validate_age(cls, value):
    if 0 < value < 100:
      return value
    else:
      raise ValueError("Enter the correct age range")


def insert_patient_details(patient: Patient):
  print(patient.name)
  print(patient.age)
  print("Inserted")

def update_patient_details(patient: Patient):
  print(patient.name)
  print(patient.age)
  print("Updated")

patient_info = {"name": "Arjun", 'email': 'ar@pwc.com', "age": '30',
                'weight': 72.0, "married": False, "allergies": ['nuts', 'milk'],
                "contact_details": {'zip': '11800', 'phone': '7893647'}}

Patient_1 = Patient(**patient_info)

insert_patient_details(Patient_1)

ARJUN
30
Inserted


How to do data validation depending on more than one field?
* Eg: If the age of patient is more than 60 then in contacts you also need emergency contact number. Here data validation depend on two field: age and contact.

**Here comes model validator**
* Now we are working with two fields (age and contact_details) so we will have to use model validator. We can't use Field validator.

In [ ]:
from pydantic import BaseModel, EmailStr, AnyUrl, field_validator, model_validator
from typing import List, Dict, Optional

class Patient(BaseModel):

  name: str
  email: Optional[EmailStr] = None
  website: Optional[AnyUrl] = None
  age: int
  weight: float
  married: Optional[bool] = False
  allergies: Optional[List[str]]= None
  contact_details: Dict[str, str]

  #Model validator.
  @model_validator(mode='after')
  def validate_emergency_contact(cls, model):
    if model.age > 60 and 'emergency' not in model.contact_details:
      raise ValueError("Pateint older then 60 require emergency contact")
    else:
      return model

  @field_validator('email')
  @classmethod
  def email_validator(cls, value):

    valid_domais = ['pwc.com', 'kymeratx.com']
    # extracting domain name from the value.
    domain_name = value.split('@')[1]    # ['arjun','pwc.com']
    if domain_name not in valid_domais:
      raise ValueError('Not a valid domain')

  @field_validator('name')
  @classmethod
  def transform_name(cls, value):
    return value.upper()

  # You can also do that using Field function. Here we want to set the limit of age usng Field validator.
  @field_validator('age', mode = 'after')
  @classmethod
  def validate_age(cls, value):
    if 0 < value < 100:
      return value
    else:
      raise ValueError("Enter the correct age range")


def insert_patient_details(patient: Patient):
  print(patient.name)
  print(patient.age)
  print("Inserted")

def update_patient_details(patient: Patient):
  print(patient.name)
  print(patient.age)
  print("Updated")

patient_info = {"name": "Arjun", 'email': 'ar@pwc.com', "age": '55',
                'weight': 72.0, "married": False, "allergies": ['nuts', 'milk'],
                "contact_details": {'zip': '11800', 'phone': '7893647'}}

Patient_1 = Patient(**patient_info)

insert_patient_details(Patient_1)

ARJUN
55
Inserted


**Computed Field**
* When you want to calculate something from the fields. Like calculating the BMI using weight and height.  

In [ ]:
from pydantic import BaseModel, EmailStr, AnyUrl, computed_field
from typing import List, Dict, Optional

class Patient(BaseModel):

  name: str
  email: Optional[EmailStr] = None
  website: Optional[AnyUrl] = None
  age: int
  weight: float
  height: float
  married: Optional[bool] = False
  allergies: Optional[List[str]]= None
  contact_details: Dict[str, str]

  @computed_field
  @property
  def calculate_bmi(self) -> float:
    bmi = round(self.weight/(self.height**2), 2)
    return bmi

def insert_patient_details(patient: Patient):
  print(patient.name)
  print(patient.age)
  print("Inserted")

def update_patient_details(patient: Patient):
  print(patient.name)
  print(patient.age)
  print(Patient(**patient_info).calculate_bmi)      # Here we are not using calculate_bmi(), we are using calculate_bmi.
  print("Updated")

patient_info = {"name": "Arjun", 'email': 'ar@gmail.com', "age": 30,
                'weight': 72.0, "height":1.72, "married": False, "allergies": ['nuts', 'milk'],
                "contact_details": {'zip': '11800', 'phone': '7893647'}}

Patient_1 = Patient(**patient_info)

update_patient_details(Patient_1)

Arjun
30
24.34
Updated


**Nested Models**

* How to use one model into another model?
* For eg. If we want to extract the zip code from the address it will cumbersome to write the code. So, instead of wrting the code what we can do is we can make a different class or model for that task.

In [ ]:
from pydantic import BaseModel

class Address(BaseModel):

  city: str
  street: str
  state: str
  pin: str

class Pateint(BaseModel):

  name: str
  age: int
  weight: float
  address: Address

address_dict = {'city': "easton", "street": "lake way", "state": "NJ", "pin":"08844"}

address1 = Address(**address_dict)

patient_info = {'name': 'Arjun', "age" : "30", "weight" : "72", "address": address1}

patient1 = Pateint(**patient_info)

print(patient1)

print(patient1.address.city)

name='Arjun' age=30 weight=72.0 address=Address(city='easton', street='lake way', state='NJ', pin='08844')
easton


Benefits of Pydantic:

1. Better organization of related data (e.g., vitals, address, insurance)

2. Reusability: Use Vitals in multiple models (e.g., Patient, MedicalRecord)

3. Readability: Easier for developers and API consumers to understand

4. Validation: Nested models are validated automatically—no extra work needed

Serialization

* How to export pydantic models as python dictionary and JSON.

In [ ]:
from pydantic import BaseModel

class Address(BaseModel):

  city: str
  street: str
  state: str
  pin: str

class Pateint(BaseModel):

  name: str
  age: int
  weight: float = 72.0
  address: Address

address_dict = {'city': "easton", "street": "lake way", "state": "NJ", "pin":"08844"}

address1 = Address(**address_dict)

patient_info = {'name': 'Arjun', "age" : "30", "weight" : "72", "address": address1}

patient1 = Pateint(**patient_info)

temp = patient1.model_dump()

print(temp)
print(type(temp))

{'name': 'Arjun', 'age': 30, 'weight': 72.0, 'address': {'city': 'easton', 'street': 'lake way', 'state': 'NJ', 'pin': '08844'}}
<class 'dict'>


If you want to export only few fields.

In [ ]:
temp_incl = patient1.model_dump(include=['name', 'age'])
print(temp_incl)
print(type(temp_incl))

{'name': 'Arjun', 'age': 30}
<class 'dict'>


Using exclude.

In [ ]:
temp_exl = patient1.model_dump(exclude=['name', 'age'])
print(temp_exl)
print(type(temp_exl))

{'weight': 72.0, 'address': {'city': 'easton', 'street': 'lake way', 'state': 'NJ', 'pin': '08844'}}
<class 'dict'>


Excluding only street from address.

In [ ]:
temp_exl = patient1.model_dump(exclude={'address':['street']})
print(temp_exl)
print(type(temp_exl))

{'name': 'Arjun', 'age': 30, 'weight': 72.0, 'address': {'city': 'easton', 'state': 'NJ', 'pin': '08844'}}
<class 'dict'>


In [ ]:
# How to not export parameters which are not defined while making objects.
# using exclude_unset. Here I don't want to include the default value of the weight which is not define below.

patient_info = {'name': 'Arjun', "age" : "30", "address": address1}
patient1 = Pateint(**patient_info)

temp_exl = patient1.model_dump(exclude_unset=True)
print(temp_exl)
print(type(temp_exl))

{'name': 'Arjun', 'age': 30, 'address': {'city': 'easton', 'street': 'lake way', 'state': 'NJ', 'pin': '08844'}}
<class 'dict'>
